In [1]:
import numpy as np
import os,math,random
from matplotlib import pyplot as plt
from scipy.stats import binned_statistic, binned_statistic_2d
from astropy.table import Table, vstack
from astropy.cosmology import FlatLambdaCDM, z_at_value
from astropy import units as u
from astropy.constants import k_B, m_p, G, M_sun
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from gdpyc import GasMap, DustMap
import time
import pandas as pd

/Users/zhengyunliang/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cosmo = FlatLambdaCDM(Om0=0.315,H0=100)

In [3]:
Cens = Table.read('../data/DESI/CensACT.fits')
Cens['R180'] = (((10**Cens['logMh']*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*(cosmo.H0**2)/(8*np.pi*G)))**(1./3)).to('kpc')/(1+Cens['z'])
Cens['R180_ang'] = Cens['R180']/cosmo.angular_diameter_distance(Cens['z']).to('kpc')

bcg_pos = SkyCoord(Cens['ra']*u.deg, Cens['dec']*u.deg)
xry_pos = SkyCoord(Cens['RA_X']*u.deg, Cens['DEC_X']*u.deg)
luw_pos = SkyCoord(Cens['ragroup']*u.deg, Cens['decgroup']*u.deg)

In [4]:
def create_circular_mask(h, w, center=None, radius=None):
    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    mask = dist_from_center <= radius
    return mask

## $2 \times 2$ bins in $M_h$ and Redshift Spaces

In [5]:
step = 0.5
mbins = np.arange(14,15.1,step)
layers = 2

r180_min = np.zeros((2,len(mbins)-1))
r180_med = np.zeros((2,len(mbins)-1))
r180_max = np.zeros((2,len(mbins)-1))
r180_min_ang = np.zeros((2,len(mbins)-1))
r180_med_ang = np.zeros((2,len(mbins)-1))
r180_max_ang = np.zeros((2,len(mbins)-1))

for zz in range(2):
    for mm in range(len(mbins)-1):
        r180_min[zz,mm] = (((10**mbins[mm]*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*cosmo.H0**2/(8*np.pi*G)))**(1./3)).to('kpc').value/(1+0.5*(zz+1))
        r180_med[zz,mm] = (((10**((mbins[mm]+mbins[mm+1])/2)*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*cosmo.H0**2/(8*np.pi*G)))**(1./3)).to('kpc').value/(1+0.5*(zz+0.5))
        r180_max[zz,mm] = (((10**(mbins[mm+1])*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*cosmo.H0**2/(8*np.pi*G)))**(1./3)).to('kpc').value/(1+0.5*zz)
        r180_min_ang[zz,mm] = r180_min[zz,mm]/cosmo.angular_diameter_distance(0.25+0.5*zz).to('kpc').value
        r180_med_ang[zz,mm] = r180_med[zz,mm]/cosmo.angular_diameter_distance(0.25+0.5*zz).to('kpc').value
        r180_max_ang[zz,mm] = r180_max[zz,mm]/cosmo.angular_diameter_distance(0.25+0.5*zz).to('kpc').value

## eROSITA Tiles Info

In [7]:
tiles = fits.open('../data/eROSITA/SKYMAPS_052022_MPE.fits')
tlt_select = Table(tiles[1].data)
# l_ = elt.transform_to('galactic').l.to('deg').value
elt_sel = SkyCoord(tlt_select['RA_CEN']*u.deg,tlt_select['DE_CEN']*u.deg)

Emin, Emax = 0.5, 2
zoom, zoom2 = 0.5, 4

idx_srv, d2d_srv, d3d_srv = xry_pos.match_to_catalog_sky(elt_sel)
srv_n = [i for i in set(tlt_select['SRVMAP'][idx_srv])]

In [30]:
srv_n.sort()

In [31]:
srv_n

[2123,
 2126,
 2129,
 2132,
 2135,
 2138,
 2141,
 2144,
 3147,
 3150,
 5120,
 5123,
 5126,
 6129,
 6132,
 6135,
 7138,
 7141,
 7144,
 8147,
 9120,
 9123,
 9126,
 9129,
 9150,
 10132,
 10135,
 11138,
 12117,
 12120,
 12123,
 12141,
 12144,
 13126,
 13129,
 13147,
 14132,
 14135,
 14150,
 15114,
 15117,
 15120,
 15138,
 16123,
 16126,
 16141,
 17129,
 17144,
 18114,
 18117,
 18132,
 19120,
 19123,
 19135,
 19147,
 20126,
 20138,
 20150,
 21111,
 21114,
 21129,
 21141,
 22117,
 22120,
 22132,
 22144,
 23123,
 23135,
 24111,
 24114,
 24126,
 24138,
 24147,
 25117,
 25129,
 25141,
 26120,
 26123,
 26132,
 26150,
 27108,
 27111,
 27126,
 27135,
 27144,
 28114,
 28117,
 28129,
 28138,
 29105,
 29120,
 29147,
 30108,
 30111,
 30123,
 30132,
 30141,
 31114,
 31126,
 31135,
 31150,
 32102,
 32105,
 32117,
 32129,
 32144,
 33108,
 33111,
 33120,
 33138,
 34114,
 34123,
 34132,
 34147,
 35102,
 35105,
 35117,
 35126,
 35135,
 35141,
 36108,
 36120,
 36129,
 37111,
 37114,
 37123,
 37138,
 37144,
 

In [9]:
len(srv_n), len(Cens)

(798, 73348)

In [10]:
def tlnm_name(tlnm_):
    if len(tlnm_) == 5:
        tlnm_ = '0'+tlnm_
    elif len(tlnm_) == 4:
        tlnm_ = '00'+tlnm_
    elif len(tlnm_) == 3:
        tlnm_ = '000'+tlnm_
    return tlnm_

def rescale_(mskk, ima_size):
    f_mc = interpolate.interp2d(np.arange(mskk.shape[1]), np.arange(mskk.shape[0]), mskk, kind='linear')
    mskk_rebin = f_mc(np.linspace(0, mskk.shape[1], ima_size), np.linspace(0, mskk.shape[1], ima_size))
    mskk_rebin[mskk_rebin>=0.5] = 1
    mskk_rebin[mskk_rebin<0.5] = 0
    return mskk_rebin

def mask_conta(tabs,detmsk,xx,yy,R_ang,xxs,yys,rrs):
    msk_id = np.ones(len(tabs))
    if len(rrs) > 0:
        mask_sources = []
        for s in range(len(rrs)):
            mask_sj = create_circular_mask(detmsk.shape[0], detmsk.shape[1], 
                                           center=((xxs[s]-xx+R_ang*2), (yys[s]-yy+R_ang*2)),
                                           radius=(rrs[s]/4+30/4))
            mask_sources.append(mask_sj)
            msk_id[np.sqrt((tabs['xx']-xxs[s])**2+(tabs['yy']-yys[s])**2)<(rrs[s]/4+30/4)] = 0
        mask_contamination = 1 - np.sum(np.array(mask_sources), axis=0)
        mask_contamination[mask_contamination < 1] = 0
    else:
        mask_contamination = np.ones((detmsk.shape[0], detmsk.shape[1]))
    return msk_id, mask_contamination

# Image Stacking

In [12]:
repeat_ = 51

pure_count = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
weighted_count = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
mask_ima = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]

pure_count_sat = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
weighted_count_sat = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
mask_ima_sat = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]

pure_count_sat_bq = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
weighted_count_sat_bq = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
mask_ima_sat_bq = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]

In [13]:
for hh in range(repeat_):
    for ii in range(layers):
        for jj in range(2):
            for kk in range(len(mbins)-1):
                ima_size = int(r180_med_ang[jj,kk]*180/np.pi*60*15/zoom)*4
                pure_count[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                weighted_count[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                mask_ima[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                pure_count_sat[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                weighted_count_sat[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                mask_ima_sat[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                pure_count_sat_bq[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                weighted_count_sat_bq[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                mask_ima_sat_bq[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))

In [ ]:
for i in range(0,len(srv_n)):
    tlnm_ = tlnm_name(str(srv_n[i]))
    if os.path.exists('../data/eROSITA/expmaps/exp_all/'+tlnm_+'_all.fits') == True:
        arf = fits.open('../eRASS1/arffiles/arf_020_ARF_00001'+tlnm_+'.fits')
        arfE = Table(arf[1].data)
        arfE['Area'] = arfE['SPECRESP']/arfE['CORRCOMB']
        ee_arf = 0.5*(arfE['ENERG_LO']+arfE['ENERG_HI'])
    else:
        print('!!!',tlnm_)
        continue
    # Counts + ExpMaps:
    fname = '../eRASS1/expmaps/exp_all/'+tlnm_+'_all.fits'
    expnm = '../eRASS1/expmaps/exp_all/'+tlnm_+'_all_exp.fits'
    im_ = fits.open(fname)[0].data
    et_ = fits.open(expnm)[0].data
    tab_ = Table(fits.open(fname)[1].data)
    det_ = (et_>0)
    w = WCS(fits.open(fname)[0].header)
    tltt = SkyCoord(tlt_select['RA_CEN'][tlt_select['SRVMAP']==int(tlnm_)]*u.deg, 
                    tlt_select['DE_CEN'][tlt_select['SRVMAP']==int(tlnm_)]*u.deg)
    gr_srv_id = np.where(tlt_select['SRVMAP'][idx_srv] == int(tlnm_))[0]
    sep_tlt = tltt.separation(mainst)
    max_ang = np.max(CenN['R180_ang'][gr_srv_id])/np.pi*180   # in deg
    ### !!!
    main_this_t = mainst[sep_tlt.to('deg').value<(2*max_ang+3.6)]
    main_this = mains[sep_tlt.to('deg').value<(2*max_ang+3.6)]
    # Image Coordinate for Each Photon
    evx, evy = w.all_world2pix(tab_['RA'], tab_['DEC'], 0)
    tab_['xx'] = evx
    tab_['yy'] = evy
    tab_['Eob'] = tab_['PI']/1000
    # Image Coordinate for Each Blind-detected Source
    emix, emiy = w.all_world2pix(main_this['RA'], main_this['DEC'], 0)
    print(i, len(gr_srv_id))
    t0 = time.time()
    for gi in range(0, len(gr_srv_id)):
        richness, redshift, mhalo = CenN['richness'][gr_srv_id[gi]], CenN['z'][gr_srv_id[gi]], CenN['logMh'][gr_srv_id[gi]]
        mi = int((CenN['logMh'][gr_srv_id[gi]]-14)/0.5)
        zi = int(redshift/0.50)
        if redshift < 0.005:
            continue
        if mi > 1:
            mi = 1
        else:
            pass
        ima_size = int(r180_med_ang[zi][mi]*180/np.pi*60*15/zoom)*4
        # Basic Info
        grpid = CenN['grpid'][gr_srv_id[gi]]
        if richness > 1:
            Sat_sel = SatN_a[(SatN_a['grpid']==grpid)] #Sat Sel
        else:
            Sat_sel = SatN_a[1:0]
        nH = np.round(np.log10(CenN['nH'][gr_srv_id[gi]].data),1)
        spec0 = Table.read('../eRASS1/nHcorr/abp'+str(int(10*nH))+'.fits')
        lum_dist = cosmo.luminosity_distance(redshift).to('cm').value
        # theta_180 in pix
        maxR = int(CenN['R180_ang'][gr_srv_id[gi]]/np.pi*180*60*15)    # in pix^2
        ratio1 = (CenN['R180'][gr_srv_id[gi]]/(ima_size/4))**2         # ? kpc^2 in one pix^2
        # Image Coordinate for BGG
        xx, yy = w.all_world2pix(CenN['RA_X'][gr_srv_id[gi]], CenN['DEC_X'][gr_srv_id[gi]], 0) # !
        xx_c, yy_c = w.all_world2pix(CenN['ra'][gr_srv_id[gi]], CenN['dec'][gr_srv_id[gi]], 0)
        xx_s, yy_s = w.all_world2pix(Sat_sel['ra'], Sat_sel['dec'], 0)
        xx, yy = np.round(xx, 0).astype('int'), np.round(yy, 0).astype('int')
        xx_c, yy_c = np.round(xx_c, 0).astype('int'), np.round(yy_c, 0).astype('int')
        xx_s, yy_s = np.round(xx_s, 0).astype('int'), np.round(yy_s, 0).astype('int')
        if (xx-maxR*2>=0)&(xx+maxR*2+1<3240)&(yy-maxR*2>=0)&(yy+maxR*2+1<3240):
            et_cut1 = et_[yy-maxR*2:yy+maxR*2, xx-maxR*2:xx+maxR*2]
        elif (xx-maxR*2>=3240)|(xx+maxR*2<0)|(yy-maxR*2>=3240)|(yy+maxR*2<0):
            print('!!!!!!')
            continue
        else:
            et_cut1 = np.zeros((maxR*4, maxR*4))
            eii_ = abs(yy-maxR*2)-np.max([yy-maxR*2,0])
            eji_ = abs(xx-maxR*2)-np.max([xx-maxR*2,0])
            eif_ = np.min([yy+maxR*2,3240])-np.max([yy-maxR*2,0])
            ejf_ = np.min([xx+maxR*2,3240])-np.max([xx-maxR*2,0])
            et_cut1[eii_:eii_+eif_, eji_:eji_+ejf_] = et_[np.max([yy-maxR*2,0]):np.min([yy+maxR*2,3240]), np.max([xx-maxR*2,0]):np.min([xx+maxR*2,3240])]
        f_et1 = interpolate.interp2d(np.arange(et_cut1.shape[1]), np.arange(et_cut1.shape[0]), et_cut1, kind='linear')
        et_cut_rebin1 = f_et1(np.linspace(0, et_cut1.shape[1], ima_size), 
                              np.linspace(0, et_cut1.shape[1], ima_size))
        detmsk1 = (et_cut1 > 0)
        detmsk_rebin1 = (et_cut_rebin1 > 0)
        tab_['Erf'] = tab_['PI']/1000*(1+redshift)
        tab_sel = tab_[(abs(xx-evx)<maxR*2)&(abs(yy-evy)<maxR*2)&(tab_['Erf']>=Emin)&(tab_['Erf']<=Emax)&(abs(tab_['xx'].astype('int')-1619.5)<1620)&(abs(tab_['yy'].astype('int')-1619.5)<1620)]
        tab_sel['texp0'] = et_[tab_sel['xx'].astype('int'), tab_sel['yy'].astype('int')]
        tab_sel['ebin'] = np.argmin((tab_sel['Eob'].data.reshape(-1,1)-rmfinfo['ENERG_LO'].data)*(tab_sel['Eob'].data.reshape(-1,1)-rmfinfo['ENERG_HI'].data), axis=1)
        tab_sel['ebinss'] = np.argmin((tab_sel['Eob'].data.reshape(-1,1)-spec0['elo'].data)*(tab_sel['Eob'].data.reshape(-1,1)-spec0['ehi'].data), axis=1)
        tab_sel['wei'] = tab_sel['Erf']/(arfE['Area'][tab_sel['ebin'].data])*4*np.pi*(lum_dist**2)/(tab_sel['texp0'])*1.6022e-9/spec0['flux'].data[tab_sel['ebinss'].data]
        tab_sel['msk'] = np.ones(len(tab_sel))
        # Mask the Contaminants
        sep_main = cenxt[gr_srv_id[gi]].separation(main_this_t)
        near_mask = np.where(((sep_main.to('arcsec').value-main_this['EXT']-30)/(maxR*4*2)<1.01)
                         &((main_this['GR']!=grpid)|(main_this['Sat']!=0)))[0] 
                            # not belong this group or satellites
        msk_id, mask_contamination1 = mask_conta(tab_sel,detmsk1,xx,yy,maxR,emix[near_mask],emiy[near_mask],(main_this['Rsrc'][near_mask]-30))
        # Mark the Satellites
        sat_sel1 = ((Sat_sel['ELG']==True)|(Sat_sel['LRG']==True)|(Sat_sel['ASKAP_idx']>0))
        msk_id1, mask_contamination01 = mask_conta(tab_sel,detmsk1,xx,yy,maxR,xx_s[sat_sel1],yy_s[sat_sel1],np.zeros(len(xx_s[sat_sel1])))
        sat_sel3 = ((Sat_sel['ELG']==False)&(Sat_sel['LRG']==False)&(Sat_sel['ASKAP_idx']<=0)&((Sat_sel['is_QSO']==True)|(Sat_sel['is_BPT']>=1)))
        msk_id3, mask_contamination03 = mask_conta(tab_sel,detmsk1,xx,yy,maxR,xx_s[sat_sel3],yy_s[sat_sel3],np.zeros(len(xx_s[sat_sel3])))
        # For X-ray Ext. Groups
        if  (CenN['pgn_'][gr_srv_id[gi]]&2**0 > 0)|(CenN['pgn_'][gr_srv_id[gi]]&2**1 > 0):
            msk_idc, mask_contaminationc = mask_conta(tab_sel,detmsk1,xx,yy,maxR,[xx_c],[yy_c],np.zeros(1))
            if (CenN['ASKAP_idx'][gr_srv_id[gi]]>0):
                msk_id1 *= msk_idc
                mask_contamination01 *= mask_contaminationc
            if (CenN['ASKAP_idx'][gr_srv_id[gi]]<=0)&((CenN['is_QSO'][gr_srv_id[gi]]==True)&(CenN['is_BPT'][gr_srv_id[gi]]>=1)):
                msk_id3 *= msk_idc
                mask_contamination03 *= mask_contaminationc
        mask_conta_rebin1 = rescale_(mask_contamination1, ima_size)
        mask_conta_rebin01 = rescale_(mask_contamination01, ima_size)
        mask_conta_rebin03 = rescale_(mask_contamination03, ima_size)
        
        if np.min(detmsk_rebin1) == True:
            masks_1 = mask_conta_rebin1*ratio1
            masks_01 = mask_conta_rebin1*mask_conta_rebin01*ratio1
            masks_03 = mask_conta_rebin1*mask_conta_rebin01*mask_conta_rebin03*ratio1
        else:
            masks_1 = mask_conta_rebin1*detmsk_rebin1*ratio1
            masks_01 = mask_conta_rebin1*mask_conta_rebin01*detmsk_rebin1*ratio1
            masks_03 = mask_conta_rebin1*mask_conta_rebin01*mask_conta_rebin03*detmsk_rebin1*ratio1
        
        if len(tab_sel) > 0:
            im2d_1, x2d, y2d, im2_ = binned_statistic_2d(tab_sel['xx'], tab_sel['yy'], msk_id, \
                bins=[np.linspace(xx-maxR*2, xx+maxR*2, ima_size+1), np.linspace(yy-maxR*2, yy+maxR*2, ima_size+1)], \
                statistic='sum')
            im2dW1, x2d, y2d, im2_ = binned_statistic_2d(tab_sel['xx'], tab_sel['yy'], tab_sel['wei']*msk_id, \
                bins=[np.linspace(xx-maxR*2, xx+maxR*2, ima_size+1),np.linspace(yy-maxR*2, yy+maxR*2, ima_size+1)], \
                statistic='sum')
            im2d_01, x2d, y2d, im2_ = binned_statistic_2d(tab_sel['xx'], tab_sel['yy'], msk_id*msk_id1, \
                bins=[np.linspace(xx-maxR*2, xx+maxR*2, ima_size+1), np.linspace(yy-maxR*2, yy+maxR*2, ima_size+1)], \
                statistic='sum')
            im2dW01, x2d, y2d, im2_ = binned_statistic_2d(tab_sel['xx'], tab_sel['yy'], tab_sel['wei']*msk_id*msk_id1, \
                bins=[np.linspace(xx-maxR*2, xx+maxR*2, ima_size+1),np.linspace(yy-maxR*2, yy+maxR*2, ima_size+1)], \
                statistic='sum')
            im2d_03, x2d, y2d, im2_ = binned_statistic_2d(tab_sel['xx'], tab_sel['yy'], msk_id*msk_id1*msk_id3, \
                    bins=[np.linspace(xx-maxR*2, xx+maxR*2, ima_size+1), np.linspace(yy-maxR*2, yy+maxR*2, ima_size+1)], \
                    statistic='sum')
            im2dW03, x2d, y2d, im2_ = binned_statistic_2d(tab_sel['xx'], tab_sel['yy'], tab_sel['wei']*msk_id*msk_id1*msk_id3, \
                    bins=[np.linspace(xx-maxR*2, xx+maxR*2, ima_size+1),np.linspace(yy-maxR*2, yy+maxR*2, ima_size+1)], \
                    statistic='sum')
        else:
            im2d_1 = np.zeros((ima_size,ima_size))
            im2dW1 = np.zeros((ima_size,ima_size))
            im2d_01 = np.zeros((ima_size,ima_size))
            im2dW01 = np.zeros((ima_size,ima_size))
            im2d_03 = np.zeros((ima_size,ima_size))
            im2dW03 = np.zeros((ima_size,ima_size))
        # seq_ = np.where(CenN['seq'][gr_srv_id[gi]]>0)[0]
        seq_ = [0]
        rdc_ = np.append(0,np.random.choice(repeat_-1,5,replace=False)+1)
        for ddd in range(len(rdc_)):
            for sss in range(len(seq_)):
                pure_count[rdc_[ddd]][seq_[sss]][zi][mi] += im2d_1.T
                weighted_count[rdc_[ddd]][seq_[sss]][zi][mi] += im2dW1.T
                mask_ima[rdc_[ddd]][seq_[sss]][zi][mi] += masks_1
                pure_count_sat[rdc_[ddd]][seq_[sss]][zi][mi] += im2d_01.T
                weighted_count_sat[rdc_[ddd]][seq_[sss]][zi][mi] += im2dW01.T
                mask_ima_sat[rdc_[ddd]][seq_[sss]][zi][mi] += masks_01
                pure_count_sat_bq[rdc_[ddd]][seq_[sss]][zi][mi] += im2d_03.T
                weighted_count_sat_bq[rdc_[ddd]][seq_[sss]][zi][mi] += im2dW03.T
                mask_ima_sat_bq[rdc_[ddd]][seq_[sss]][zi][mi] += masks_03
    t7 = time.time()
    print(i, len(gr_srv_id), t7-t0, (t7-t0)/len(gr_srv_id))
    
    if ((i+1)%100 == 0)or(i == len(srv_n)-1):
        for ss in range(layers):
            for mmj in range(2):
                hud_list = [fits.PrimaryHDU()]
                hud_list1 = [fits.PrimaryHDU()]
                hud_list2 = [fits.PrimaryHDU()]
                hud_list3 = [fits.PrimaryHDU()]
                for zzj in range(2):
                    for hh in range(repeat_):
                        hud_list.append(fits.ImageHDU(pure_count[hh][ss][zzj][mmj]))
                        hud_list.append(fits.ImageHDU(weighted_count[hh][ss][zzj][mmj]))
                        hud_list.append(fits.ImageHDU(mask_ima[hh][ss][zzj][mmj]))
                        hud_list1.append(fits.ImageHDU(pure_count_sat[hh][ss][zzj][mmj]))
                        hud_list1.append(fits.ImageHDU(weighted_count_sat[hh][ss][zzj][mmj]))
                        hud_list1.append(fits.ImageHDU(mask_ima_sat[hh][ss][zzj][mmj]))
                        hud_list2.append(fits.ImageHDU(pure_count_sat_bq[hh][ss][zzj][mmj]))
                        hud_list2.append(fits.ImageHDU(weighted_count_sat_bq[hh][ss][zzj][mmj]))
                        hud_list2.append(fits.ImageHDU(mask_ima_sat_bq[hh][ss][zzj][mmj]))
                hdul = fits.HDUList(hud_list)
                hdul1 = fits.HDUList(hud_list1)
                hdul2 = fits.HDUList(hud_list2)
                if (i+1)%100 == 0:
                    hdul.writeto('./stack_spec/specinfo/'+str(ss)+'/stim_'+str(mmj)+'_med'+str(i)+'.fits', overwrite=True)
                    hdul1.writeto('./stack_spec/specinfo/'+str(ss)+'/stim_'+str(mmj)+'_med'+str(i)+'_sats.fits', overwrite=True)
                    hdul2.writeto('./stack_spec/specinfo/'+str(ss)+'/stim_'+str(mmj)+'_med'+str(i)+'_sats_bq.fits', overwrite=True)
                elif i == len(srv_n)-1:
                    hdul.writeto('./stack_spec/specinfo/'+str(ss)+'/stim_'+str(mmj)+'.fits', overwrite=True)
                    hdul1.writeto('./stack_spec/specinfo/'+str(ss)+'/stim_'+str(mmj)+'_sats.fits', overwrite=True)
                    hdul2.writeto('./stack_spec/specinfo/'+str(ss)+'/stim_'+str(mmj)+'_sats_bq.fits', overwrite=True)